# IBM Data Science Capstone Assignment 2 Part 1: Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment - **DONE**
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe - **DONE, see below**


I used this link as the reference for scrapping a table from a website: https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [1]:
# Import necessary libraries
import requests
import lxml.html as lh
import pandas as pd

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

This means that there are 3 columns per row

In [4]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


#### Creating Pandas DataFrame

Each header is appended to a tuple along with an empty list.

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

[181, 181, 181]

This shows that each of the 3 columns has exactly 181 rows

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

3. To create the referenced dataframe from the assignment:

- The dataframe will consist of three columns: **PostalCode**, **Borough**, and **Neighborhood** - **DONE**

In [8]:
# Access the top 5 rows of the data frame 
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [9]:
# Remove \n sign in the table
df = df.replace('\n','', regex=True)
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
# Renaming the columns header
df.columns = ['PostalCode', 'Borough', 'Neighbourhood']

cols = df.columns.tolist()
cols

df = df[cols]

df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


- Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned** - **DONE**

In [11]:
# Drop the rows of Borough that has Not assigned value in it
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

df = df.reset_index(drop=True)

df

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma.

This instruction is outdated as the neighbourhood is already combined from Wikipedia - **DONE**

- If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough

Again, this instruction is also outdated as for each Not assigned value in Borough, it is always Not assigned in Neighbourhood, hence the rows is also removed in the previous step - **DONE**

In [12]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making - **DONE**

In [13]:
# Remove the last row of the table as it does not contain the Postalcode, Borough, and Neighbourhood information
df.drop([103], inplace = True)
df

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe - **DONE**

In [14]:
# Check the shape of the data frame
df.shape

(103, 3)

4.Submit a link to your Notebook on your Github repository. (10 marks) - **DONE**

In [17]:
# Export dataframe to csv for the part of the assignment
df.to_csv('assignment2_part1.csv',index=False)